# Changes to Labor Market Racial Gap

Request BLS data on the unemployment rate and total employment for black, Hispanic, and white American, including youth (age 16-19). Examine the two year change in these figures. 

In [2]:
# Preliminaries (Python 2.7)
import requests
import json
import config # file called config.py with my API key
import pandas as pd

import math

%matplotlib inline

#### Set the date range 

Provide a start and end year to be used by the API.

In [16]:
# Include the start and end year here
date_range = (2015, 2017)

# Divide the date range into BLS-API-friendly length requests
req_no = int(math.ceil((date_range[1] - date_range[0]) / 10.0))
dates = []
for i in range(0,req_no):
    d1 = str(date_range[0]+i*10)
    d2 = str(date_range[0]+i*10+9)
    dates.append((d1,d2))
dates[-1] = (dates[-1][0], str(date_range[1]))

dates

[('2015', '2017')]

#### API and series information

Identify the series of interest and add headers to indicate json data should be read.

In [44]:
# URL, key, and headers same as above
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
key = '?registrationkey={}'.format(config.bls_key)
headers = {'Content-type': 'application/json'}

# This time, store the series into a dictionary with description
series_dict = {'LNS12000000': 'Overall Employment',
               'LNS14000000': 'Overall Employment Rate',
               'LNS12000006': 'Employment - Black',
               'LNS12000009': 'Employment - Hispanic',
               'LNS12000003': 'Employment - White',
               'LNS14000006': 'Unemp Rate - Black',
               'LNS14000009': 'Unemp Rate - Hispanic',
               'LNS14000003': 'Unemp Rate - White',
               'LNS12000018': 'Employment - Black - 16-19',
               'LNS12000021': 'Employment - Hispanic - 16-19',
               'LNS12000015': 'Employment - White - 16-19',
               'LNS14000018': 'Unemp Rate - Black - 16-19',
               'LNS14000021': 'Unemp Rate - Hispanic - 16-19',
               'LNS14000015': 'Unemp Rate - White - 16-19'}

df = pd.DataFrame()  # blank pandas dataframe to be filled later

#### Make requests for each date range in dates

In [45]:
for start, end in dates: 
    
    # The data sent in the post request now includes a start and end year
    data = json.dumps({"seriesid":series_dict.keys(), "startyear":start, "endyear":end})
    p = requests.post('{}{}'.format(url, key), headers=headers, data=data).json()
    d = {} # New dictionary to be filled with data
    for series in p['Results']['series']:
        s = series['seriesID']  # Shorten name to 's'
        
        # Add dictionary entry with series and reverse series order
        d[s] = pd.DataFrame(series['data']).iloc[::-1]
        if len(series['data']) > 0:  # This if/else is to allow for series of different lengths
            
            # Convert BLS API dates to readable format (YYYY-MM-DD)
            d[s]['date'] = pd.to_datetime(d[s]['period'] + ' ' + d[s]['year'])
            
            # Keep only date and series values
            d[s] = d[s].set_index('date')['value'].astype(float)
            
            # Rename and identify values as floating point numbers
            d[s] = d[s].rename(series_dict[s])
        else:  # If blank, leave as a blank pandas series
            d[s]['date'] = d[s]['value'] = pd.Series()
            d[s] = d[s]['value'].rename(series_dict[s])
            
    # Combine the dataframes for each range of years into one by appending        
    df = df.append(pd.concat([d[k] for k in series_dict.keys()], axis=1))

In [49]:
df.loc['2017-06-01']

Unemp Rate - Hispanic                 4.8
Unemp Rate - White - 16-19           11.7
Unemp Rate - Black - 16-19           21.1
Unemp Rate - Black                    7.1
Overall Employment Rate               4.4
Unemp Rate - White                    3.8
Unemp Rate - Hispanic - 16-19        13.6
Employment - Black                18576.0
Employment - Black - 16-19          611.0
Employment - Hispanic             25974.0
Employment - White               120091.0
Overall Employment               153168.0
Employment - Hispanic - 16-19      1110.0
Employment - White - 16-19         4105.0
Name: 2017-06-01 00:00:00, dtype: float64

In [ ]:
"<b>Hispanic, age 16-19:</b><br>June 2015: 20.7%<br>June 2017: 13.6%<br>Change: -7.1"

#### View the results

In [21]:
df.diff(24).tail(1)

,Unemp Rate - Hispanic,Employment - White - 16-19,Unemp Rate - White - 16-19,Unemp Rate - Black - 16-19,Unemp Rate - Black,Overall Employment Rate,Unemp Rate - White,Employment - Hispanic - 16-19,Overall Employment,Employment - White,Employment - Black,Employment - Hispanic,Employment - Black - 16-19,Unemp Rate - Hispanic - 16-19
date,,,,,,,,,,,,,,
2017-06-01,-1.9,146.0,-3.9,-10.2,-2.5,-0.9,-0.8,107.0,3152.0,1266.0,619.0,1133.0,49.0,-7.1


In [47]:
df.pct_change(24).tail(1)

,Unemp Rate - Hispanic,Unemp Rate - White - 16-19,Unemp Rate - Black - 16-19,Unemp Rate - Black,Overall Employment Rate,Unemp Rate - White,Unemp Rate - Hispanic - 16-19,Employment - Black,Employment - Black - 16-19,Employment - Hispanic,Employment - White,Overall Employment,Employment - Hispanic - 16-19,Employment - White - 16-19
date,,,,,,,,,,,,,,
2017-06-01,-0.283582,-0.25,-0.325879,-0.260417,-0.169811,-0.173913,-0.342995,0.059609,0.226908,0.063811,0.018264,0.029756,0.189711,0.079979


### Plotly plots

In [24]:
import plotly.plotly as py
from plotly.graph_objs import *
py.sign_in('bdew', config.plotly_key)

trace1 = {
  "x": ["White", "Black", "Hispanic"], 
  "y": [-0.8, -2.5, -1.9], 
  "hoverinfo": "text", 
  "marker": {"color": "rgb(0, 0, 102)"}, 
  "name": "All ages", 
  "text": ["<b>White, all ages:</b><br>June 2015: 4.6%<br>June 2017: 3.8%<br>Change: -0.8", "<b>Black, all ages:</b><br>June 2015: 9.6%<br>June 2017: 7.1%<br>Change: -2.5", "<b>Hispanic, all ages:</b><br>June 2015: 6.7%<br>June 2017: 4.8%<br>Change: -1.9"], 
  "textsrc": "bdew:69:693ae1", 
  "type": "bar", 
  "uid": "181fdc", 
  "visible": True, 
  "xsrc": "bdew:69:12a98e", 
  "ysrc": "bdew:69:90b239"
}
trace2 = {
  "x": ["White", "Black", "Hispanic"], 
  "y": [-3.9, -10.2, -7.1], 
  "hoverinfo": "text", 
  "marker": {"color": "rgb(153, 204, 255)"}, 
  "name": "Youth (16-19)", 
  "text": ["<b>White, age 16-19:</b><br>June 2015: 15.6%<br>June 2017: 11.7%<br>Change: -3.9", "<b>Black, age 16-19:</b><br>June 2015: 31.3%<br>June 2017: 21.1%<br>Change: -10.2", "<b>Hispanic, age 16-19:</b><br>June 2015: 20.7%<br>June 2017: 13.6%<br>Change: -7.1"], 
  "textsrc": "bdew:69:f2e4bb", 
  "type": "bar", 
  "uid": "fad4e8", 
  "xsrc": "bdew:69:12a98e", 
  "ysrc": "bdew:69:d254fe"
}
data = Data([trace1, trace2])
layout = {
  "annotations": [
    {
      "x": -0.13, 
      "y": 1.27444444444, 
      "font": {"size": 16}, 
      "showarrow": False, 
      "text": "<b>Unemployment Rate Change </b>", 
      "xref": "paper", 
      "yref": "paper"
    }, 
    {
      "x": -0.11, 
      "y": 1.19333333333, 
      "font": {"size": 13}, 
      "showarrow": False, 
      "text": "<i>Change in U6 unemployment rate, June 2015-June 2017</i>", 
      "xref": "paper", 
      "yref": "paper"
    }, 
    {
      "x": -0.129787234043, 
      "y": -0.0942307692308, 
      "align": "left", 
      "showarrow": False, 
      "text": "http://cepr.net/<br>Data source: Bureau of Labor Statistics, Current Employment Statistics", 
      "xref": "paper", 
      "yref": "paper"
    }, 
    {
      "x": -0.2, 
      "y": -1.13928571429, 
      "showarrow": False, 
      "text": "-0.8", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 0.176595744681, 
      "y": -4.4, 
      "showarrow": False, 
      "text": "-3.9", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 0.795744680851, 
      "y": -2.90714285714, 
      "showarrow": False, 
      "text": "-2.5", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 1.18510638298, 
      "y": -10.5678571429, 
      "showarrow": False, 
      "text": "-10.2", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 1.77234042553, 
      "y": -2.27857142857, 
      "showarrow": False, 
      "text": "-1.9", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 2.17446808511, 
      "y": -7.46428571429, 
      "showarrow": False, 
      "text": "-7.1", 
      "xref": "x", 
      "yref": "y"
    }
  ], 
  "autosize": False, 
  "bargroupgap": 0.12, 
  "barmode": "group", 
  "height": 400, 
  "legend": {
    "x": 0.0126243489781, 
    "y": 0.270054945055
  }, 
  "margin": {
    "r": 10, 
    "t": 90, 
    "b": 40, 
    "l": 70
  }, 
  "showlegend": True, 
  "title": "", 
  "width": 550, 
  "xaxis": {
    "anchor": "y", 
    "autorange": False, 
    "range": [-0.5, 2.5], 
    "showticklabels": True, 
    "side": "top", 
    "tickfont": {
      "color": "rgb(68, 68, 68)", 
      "size": 14
    }, 
    "ticks": "", 
    "title": "", 
    "type": "category"
  }, 
  "yaxis": {
    "autorange": False, 
    "range": [-11, 0], 
    "side": "left", 
    "title": "Percentage points", 
    "titlefont": {"color": "rgb(127, 127, 127)"}, 
    "type": "linear"
  }
}
fig = Figure(data=data, layout=layout)
plot_url = py.plot(fig, filename="Unemployment Rate Changes")

In [53]:
import plotly.plotly as py
from plotly.graph_objs import *
py.sign_in('bdew', config.plotly_key)

trace1 = {
  "x": ["White", "Black", "Hispanic"], 
  "y": [1.8, 6.0, 6.4], 
  "hoverinfo": "text", 
  "marker": {"color": "rgb(0, 0, 102)"}, 
  "name": "All ages", 
  "text": ["<b>White, all ages:</b><br>June 2015: 117,937,000<br>June 2017: 120,091,000<br>Total growth: 1.8%", "<b>Black, all ages:</b><br>June 2015: 17,531,000<br>June 2017: 18,576,000<br>Total growth: 6.0%", "<b>Hispanic, all ages:</b><br>June 2015: 24,416,000<br>June 2017: 25,974,000<br>Total growth: 6.4%"], 
  "textsrc": "bdew:71:e9680d", 
  "type": "bar", 
  "uid": "181fdc", 
  "visible": True, 
  "xsrc": "bdew:71:65122f", 
  "ysrc": "bdew:71:3baf3c"
}
trace2 = {
  "x": ["White", "Black", "Hispanic"], 
  "y": [8.0, 22.7, 19.0], 
  "hoverinfo": "text", 
  "marker": {"color": "rgb(153, 204, 255)"}, 
  "name": "Youth (16-19)", 
  "text": ["<b>White, age 16-19:</b><br>June 2015: 3,801,000<br>June 2017: 4,105,000<br>Total growth: 8.0%", "<b>Black, age 16-19:</b><br>June 2015: 498,000<br>June 2017: 611,000<br>Total growth: 22.7%", "<b>Hispanic, age 16-19:</b><br>June 2015: 933,000<br>June 2017: 1,110,000<br>Total growth: 19.0%"], 
  "textsrc": "bdew:71:c6f458", 
  "type": "bar", 
  "uid": "fad4e8", 
  "xsrc": "bdew:71:65122f", 
  "ysrc": "bdew:71:9f5fba"
}
data = Data([trace1, trace2])
layout = {
  "annotations": [
    {
      "x": -0.13, 
      "y": 1.15521367521, 
      "font": {"size": 16}, 
      "showarrow": False, 
      "text": "<b>Employment Growth</b>", 
      "xref": "paper", 
      "yref": "paper"
    }, 
    {
      "x": -0.11, 
      "y": 1.06641025641, 
      "font": {"size": 13}, 
      "showarrow": False, 
      "text": "<i>Percent change in number of people employed, June 2015—June 2017</i>", 
      "xref": "paper", 
      "yref": "paper"
    }, 
    {
      "x": -0.129787234043, 
      "y": -0.209615384615, 
      "align": "left", 
      "showarrow": False, 
      "text": "http://cepr.net/<br>Data source: Bureau of Labor Statistics, Current Employment Statistics", 
      "xref": "paper", 
      "yref": "paper"
    }, 
    {
      "x": -0.2, 
      "y": 2.6, 
      "showarrow": False, 
      "text": "1.8%", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 0.176595744681, 
      "y": 8.8, 
      "showarrow": False, 
      "text": "8.0%", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 0.795744680851, 
      "y": 6.8, 
      "showarrow": False, 
      "text": "6.0%", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 1.18510638298, 
      "y": 23.5, 
      "showarrow": False, 
      "text": "22.7%", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 1.8, 
      "y": 7.2, 
      "showarrow": False, 
      "text": "6.4%", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 2.17446808511, 
      "y": 19.8, 
      "showarrow": False, 
      "text": "19.0%", 
      "xref": "x", 
      "yref": "y"
    }
  ], 
  "autosize": False, 
  "bargroupgap": 0.12, 
  "barmode": "group", 
  "height": 400, 
  "legend": {
    "x": 0.0126243489781, 
    "y": 0.8
  }, 
  "margin": {
    "r": 10, 
    "t": 50, 
    "b": 70, 
    "l": 70
  }, 
  "showlegend": True, 
  "title": "", 
  "width": 550, 
  "xaxis": {
    "anchor": "y", 
    "autorange": False, 
    "range": [-0.5, 2.5], 
    "showticklabels": True, 
    "side": "bottom", 
    "tickfont": {
      "color": "rgb(68, 68, 68)", 
      "size": 14
    }, 
    "ticks": "", 
    "title": "", 
    "type": "category"
  }, 
  "yaxis": {
    "autorange": False, 
    "range": [0, 24.9], 
    "ticksuffix": "%",
    "side": "left", 
    "title": "Percent change", 
    "titlefont": {"color": "rgb(127, 127, 127)"}, 
    "type": "linear"
  }
}
fig = Figure(data=data, layout=layout)
plot_url = py.plot(fig, filename="Employment Changes")